In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
from google.colab import files
from sklearn.model_selection import train_test_split
import tensorflow as tf
import io
sns.set

<function seaborn.rcmod.set>

In [0]:
uploaded = files.upload()

Saving Iris.csv to Iris (1).csv


# 変数を出力させる方法

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(y)

TensorFlowでは、tf.Sessionを通して計算グラフを構築して実行しなければなりません。また、tf.global_variables_initializerを使用して、計算グラフ内の変数を初期化する必要があります

#計算グラフの構成要素
学習対象のパラメータを定数ではなく、変数としておくことで、パラメーの更新、学習がある。

In [0]:
#変数(可変)
a = tf.Variable(1,name='a')
#定数(不変)
b = tf.constant(1,name='b')
#値を代入して、代入した結果を返す
#aにa+bを代入して代入後のaの値を返す
c = tf.assign(a,a+b)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print('一回目:[c,a]=',sess.run([c,a]))
  #変数cが更新されている
  print('二回目:[c,a]=',sess.run([c,a]))

tf.assignは値を代入して、代入して、代入して結果を返す操作を表す
aは変数なので更新できるが、bは定数なので更新できない
tf.global_variables_initializerは全ての変数を初期化する、操作をしている。
変数を利用する場合は、セッションのはじめに必ず初期化。

#  セッション
変数はセッションごとに初期化が必要、あるセッションの中で変数を更新したとしても、別のセッションでは、変数の更新結果はセッションをまたいで引き継がれない
セッションが変わると、変数が初期化される。
学習対象のパラメータを変数として定義すると、同一セッションを維持している間しか、学習後の結果を利用できない。

# 最適化と勾配法
最急降下法
1パタメータを適当な値で初期化  
2与えられたパラメータに置ける関数の傾きを計算  
3もっとも傾きの大きい方向に、パラメータを少しずらす  
42,3を繰り返す  


In [0]:
#パラメータは変数として定義
x = tf.Variable(0.,name='x')
#パラメータを使って最小化したい関数を定義
func = (x-1) **2
#learning_rateは一度にずらす大きさを決める
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate = 0.1
)
#xを少しずらす操作を表す
train_step = optimizer.minimize(func)
#trai_stepを繰り返し実行する
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20):
    sess.run(train_step)
  print('x = ',sess.run(x))

In [0]:
from keras.datasets import boston_housing

In [0]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

# 標準化及び正規化

In [0]:
import scipy.stats
x_train = scipy.stats.zscore(x_train)
x_test = scipy.stats.zscore(x_test)

In [0]:
x_train

In [0]:
y_train = np.log(y_train)

In [0]:
y_test = np.log(y_test)

In [0]:
print(x_train.shape)
print(y_train.shape)

In [0]:
%matplotlib inline 
import matplotlib.pyplot as plt

# モデルの定義

In [0]:
#説明変数用のプレースホルダー
x = tf.placeholder(tf.float32,(None,13),name='x')
#正解データ用のプレースホルダ
y = tf.placeholder(tf.float32,(None,1),name='y')

#説明変数を重みwで足し合わせただけのモデル
w = tf.Variable(tf.random_normal((13,1)))
pred = tf.matmul(x,w)

In [0]:
print(x)
print(y)
print(pred)

# 最適化　損失関数の定義
最適化のステップはtf.train.GradientDecentOptimizerを使う


In [0]:
#実測値と推定値の差の２乗の平均をとる

In [0]:
loss = tf.reduce_mean((y-pred)**2)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)
train_step = optimizer.minimize(loss)

In [0]:
print(x_train.shape)
print(y_train.shape)

In [0]:
print(x_train.shape)
print(y_train.shape)


In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for step in range(30):
    #train_stepがNoneを返すので、_で受けて置く
    train_loss,_ = sess.run([loss,train_step],feed_dict={x:x_train,y:y_train.reshape((-1,1))})
    print('loss:',train_loss)         
#     print('step:{},train_loss:{}'.format(step,train_loss))
       #学習が終わったら、評価用のデータに対して予測してみる
                         

#回帰 HOUSE PRICE

In [7]:
#データのアップロード
uploaded = files.upload()

Saving test.csv to test.csv


In [8]:
uploaded = files.upload()

Saving train.csv to train.csv


In [0]:
#データの読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
y = train.SalePrice[:,np.newaxis]
X = train.loc[:,[ 'GrLivArea','YearBuilt']].values

In [0]:
#目的変数yが右に裾の分布なので対数変換
y = np.log(y)

In [0]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print( y_train.shape)

(934, 2)
(934, 1)
(234, 2)
(934, 1)


In [24]:


class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
      
        # X・y:Numpyで受け取る
        
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.0001
batch_size = 10
num_epochs = 100

n_hidden1 = 200
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
#tf.placeholderでテンソルを受付られるようにするにはshapeで引数を指定
#未知の次元方向についてはNone
#計算グラフの関数の引数
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    #重みの初期化
    #tf.Variable計算グラフ上の変数のシンボル
    #正規分布でランダムな値を出力する
    #ここではshapeを入力する
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
  #tf.add足し算、tf.matmul掛け算　、重みとバイアスは辞書を参照
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    #活性化関数に全結合層
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return layer_output

# ネットワーク構造の読み込み    
#
logits = example_net(X)

# 目的関数
#目的関数として、
#サンプルあたりのlossを計算
#Yが正解ラベル、
loss_op = tf.reduce_mean(tf.losses.mean_squared_error(labels=Y,predictions=logits))
# 最適化手法、,adamを使用して、学習率を引数に渡す
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss_op)

# 推定結果
#1 と　0の２値に分類するための操作
#-0.5で閾値を超えたどうかで正負を判定している
# tf.sign(x, name=None), 要素ごとに正なら1、負なら0
correct_pred = logits
# 指標値計算
mse = tf.reduce_mean(tf.losses.mean_squared_error(labels=Y,predictions=logits))

# variableの初期化
init = tf.global_variables_initializer()

debug  = tf.sign(Y - 0.5)

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        #train_op 最適化手法、損失関数
        

        #np.ceil# 切り上げ (大きい側の整数に丸める)
        #バッチ処理する回数（全体のサンプル数/バッチサイズ)
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_mse = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #学習
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #セッションの引数
            loss_, mse_,pred_ = sess.run([loss_op, mse,logits], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss_
            total_mse += mse_
        total_loss /= n_samples
#    total_mse /= n_samples
        val_loss, val_mse = sess.run([loss_op, mse], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss_, val_loss,))
    test_mse = sess.run(mse, feed_dict={X: X_test, Y: y_test})
    print( y_test)
    print("test_mse : {:.3f}".format(test_mse))
   
    print('pred:',pred_)

Epoch 0, loss : 1357396352.0000, val_loss : 1171160448.0000
Epoch 1, loss : 659897728.0000, val_loss : 552183104.0000
Epoch 2, loss : 352399488.0000, val_loss : 256119216.0000
Epoch 3, loss : 183273872.0000, val_loss : 122349944.0000
Epoch 4, loss : 99896544.0000, val_loss : 65052064.0000
Epoch 5, loss : 55888756.0000, val_loss : 37589264.0000
Epoch 6, loss : 26994426.0000, val_loss : 20497878.0000
Epoch 7, loss : 13033373.0000, val_loss : 11537931.0000
Epoch 8, loss : 7400080.5000, val_loss : 7431818.5000
Epoch 9, loss : 5098904.5000, val_loss : 5107247.0000
Epoch 10, loss : 3743653.2500, val_loss : 3686319.5000
Epoch 11, loss : 2797071.2500, val_loss : 2774070.2500
Epoch 12, loss : 2123839.0000, val_loss : 2162641.2500
Epoch 13, loss : 1637026.2500, val_loss : 1735578.0000
Epoch 14, loss : 1275674.3750, val_loss : 1423635.7500
Epoch 15, loss : 1005850.1250, val_loss : 1191007.1250
Epoch 16, loss : 831418.2500, val_loss : 1016609.4375
Epoch 17, loss : 702923.5625, val_loss : 878457.68

In [0]:
# #
# BATCH_SIZE = 32
# step = 0
# get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# with tf.Session() as sess:
#   sess.run(tf.global_variables_initializer())
#   #100エポック回す
#   for epoch in range(10):
#     for x_batch,y_batch in get_mini_batch_train:
#       train_loss,_ = sess.run([loss,train_step],feed_dict={x:x_batch, y:y_batch.reshape((-1,1))})
#       print('step:{},train_loss:{}'.format(step,train_loss))
#       step +=1

# 分類問題 MNIST

In [26]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [27]:
X_train = x_train.astype(np.float)
X_test = x_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [0]:
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [29]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_val_one_hot = enc.fit_transform(y_val[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
y_val = enc.fit_transform(y_val)
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) #

(48000,)
(48000, 10)
float64


In [0]:
y_val_one_hot = enc.fit_transform(y_train[:, np.newaxis])

In [31]:
X_train.shape

(48000, 28, 28)

In [32]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [0]:
X_train =X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)
X_val = X_val.reshape(-1,784)

In [35]:
X_test.shape

(10000, 784)

In [0]:
y_train = y_train[:,np.newaxis]
y_test = y_test[:,np.newaxis]
y_val = y_val[:,np.newaxis]

In [41]:
y_val 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [42]:
X_train.shape

(48000, 784)

In [43]:
y_train.shape

(48000, 1)

In [44]:
y_train.shape

(48000, 1)

In [45]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
      
        # X・y:Numpyで受け取る
        
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.0001
batch_size = 100
num_epochs = 10

n_hidden1 = 200
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
#tf.placeholderでテンソルを受付られるようにするにはshapeで引数を指定
#未知の次元方向についてはNone
#計算グラフの関数の引数
X = tf.placeholder("float", [None,n_input])
Y = tf.placeholder("float", [None,n_classes ])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot , batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    #重みの初期化
    #tf.Variable計算グラフ上の変数のシンボル
    #正規分布でランダムな値を出力する
    #ここではshapeを入力する
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
  #tf.add足し算、tf.matmul掛け算　、重みとバイアスは辞書を参照
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    #活性化関数に全結合層
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    layer_output = layer_3 
    return layer_output

# ネットワーク構造の読み込み    

logits = example_net(X)

# 目的関数
#目的関数として、
#サンプルあたりのlossを計算
#Yが正解ラベル、
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法、,adamを使用して、学習率を引数に渡す
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss_op)

# 推定結果
# 軸の指定が必要

correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(logits,1))
# correct_pred = logits
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()



# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        #train_op 最適化手法、損失関数
        

        #np.ceil# 切り上げ (大きい側の整数に丸める)
        #バッチ処理する回数（全体のサンプル数/バッチサイズ)
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #学習
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            
            #セッションの引数
            loss,acc = sess.run([loss_op, accuracy ], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
  
            total_acc += acc
        total_loss /= n_samples

        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_loss,test_acc = sess.run([loss_op, accuracy], feed_dict={X: X_test, Y:y_test_one_hot })
    
    print("test_acc : {:.3f}".format(test_acc))
    
#     print('pred:',pred_)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0, loss : 57772.3164, val_loss : 56439.4062, acc : 0.470, val_acc : 0.363
Epoch 1, loss : 36475.6367, val_loss : 33263.4141, acc : 0.550, val_acc : 0.528
Epoch 2, loss : 26066.0566, val_loss : 23327.5195, acc : 0.630, val_acc : 0.618
Epoch 3, loss : 20049.0332, val_loss : 18010.9688, acc : 0.680, val_acc : 0.676
Epoch 4, loss : 15635.4053, val_loss : 14668.6826, acc : 0.710, val_acc : 0.717
Epoch 5, loss : 13022.9150, val_loss : 12382.7012, acc : 0.760, val_acc : 0.751
Epoch 6, loss : 11424.6201, val_loss : 10756.3916, acc : 0.790, val_acc : 0.772
Epoch 7, loss : 10138.4756, val_loss : 9517.8945, acc : 0.800, val_acc : 0.788
Epoch 8, loss : 9218.7031, val_loss : 8563.4434, acc : 0.800, val_acc : 0.802
Epoch 9, loss : 8366.1982, val_loss : 7797.3389, acc : 0.800, val_acc : 0.813
test_acc : 0.8

In [0]:
#  iris

# 分類  Iris

In [46]:
from google.colab import files
import io
uploaded = files.upload()
df = pd.read_csv(io.StringIO(uploaded['Iris.csv'].decode('utf-8')))

Saving Iris.csv to Iris (3).csv


In [0]:
y = df["Species"]


In [0]:
 #　One-Hot Encoding
  y = pd.get_dummies(y)

In [61]:
y

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [0]:
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

In [0]:
y = np.array(y)
X = np.array(X)

In [64]:
y.shape

(150, 3)

In [0]:
y = y.astype(np.int)[:, np.newaxis]

In [0]:
y= y.reshape(-1,3)

In [66]:
X.shape

(150, 4)

In [0]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [0]:
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [0]:
y.shape

(150, 3)

In [68]:
y.shape

(150, 3)

In [73]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
      
        # X・y:Numpyで受け取る
        y = np.array(y)
        X = np.array(X)
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 100
num_epochs = 100

n_hidden1 = 200
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
#tf.placeholderでテンソルを受付られるようにするにはshapeで引数を指定
#未知の次元方向についてはNone
#計算グラフの関数の引数
X = tf.placeholder("float", [None,n_input])
Y = tf.placeholder("float", [None,n_classes ])

# trainのミニバッチイテレータ

get_mini_batch_train = GetMiniBatch(X_train, y_train , batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    #重みの初期化
    #tf.Variable計算グラフ上の変数のシンボル
    #正規分布でランダムな値を出力する
    #ここではshapeを入力する
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
  #tf.add足し算、tf.matmul掛け算　、重みとバイアスは辞書を参照
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    #活性化関数に全結合層
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    layer_output = layer_3 
    return layer_output

# ネットワーク構造の読み込み    

logits = example_net(X)

# 目的関数
#目的関数として、
#サンプルあたりのlossを計算
#Yが正解ラベル、
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法、,adamを使用して、学習率を引数に渡す
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss_op)

# 推定結果
# 軸の指定が必要

correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(logits,1))
# correct_pred = logits
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()



# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        #train_op 最適化手法、損失関数
        

        #np.ceil# 切り上げ (大きい側の整数に丸める)
        #バッチ処理する回数（全体のサンプル数/バッチサイズ)
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #学習
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            
            #セッションの引数
            loss,acc = sess.run([loss_op, accuracy ], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
  
            total_acc += acc
        total_loss /= n_samples

        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_loss,test_acc = sess.run([loss_op, accuracy], feed_dict={X: X_test, Y:y_test })
    
    print("test_acc : {:.3f}".format(test_acc))
    

Epoch 0, loss : 153.6082, val_loss : 172.9358, acc : 0.365, val_acc : 0.375
Epoch 1, loss : 135.9296, val_loss : 154.5746, acc : 0.385, val_acc : 0.375
Epoch 2, loss : 118.8212, val_loss : 136.2441, acc : 0.427, val_acc : 0.375
Epoch 3, loss : 104.7423, val_loss : 118.0798, acc : 0.521, val_acc : 0.375
Epoch 4, loss : 92.2951, val_loss : 101.3018, acc : 0.573, val_acc : 0.458
Epoch 5, loss : 81.1569, val_loss : 86.9580, acc : 0.615, val_acc : 0.583
Epoch 6, loss : 70.4543, val_loss : 75.1342, acc : 0.646, val_acc : 0.583
Epoch 7, loss : 59.7432, val_loss : 63.2562, acc : 0.667, val_acc : 0.583
Epoch 8, loss : 48.8090, val_loss : 51.4209, acc : 0.646, val_acc : 0.542
Epoch 9, loss : 37.6368, val_loss : 39.7368, acc : 0.635, val_acc : 0.542
Epoch 10, loss : 26.0842, val_loss : 27.8676, acc : 0.635, val_acc : 0.542
Epoch 11, loss : 14.7790, val_loss : 15.9291, acc : 0.667, val_acc : 0.542
Epoch 12, loss : 5.7170, val_loss : 7.5369, acc : 0.771, val_acc : 0.708
Epoch 13, loss : 2.0277, val